# What is Pandas?
One of the best options for working with tabular data in Python is to use the [Python Data Analysis Library](http://pandas.pydata.org/) (a.k.a. Pandas). The Pandas library provides data structures, produces high quality plots with [matplotlib](http://matplotlib.org/) and integrates nicely with other libraries that use [NumPy](http://www.numpy.org/) arrays.

Like `NumPy`, Pandas introduces some new data types to our Python world, the most important of which is the **DataFrame**. 

Briefly, a DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. It is similar to a spreadsheet or an SQL table or the data.frame in R. A DataFrame always has an index (0-based). An index refers to the position of an element in the data structure.

Once we load our data into a DataFrame, pandas gives a host of analytical capabilities with the data. Among these are:
  - Generate quick data summaries and data descriptions
  - Subset/select/query specific rows and/or columns of data
  - Handle missing data
  - Group/aggregate data based on one or more common attributes
  - Sort, transform, pivot, and "melt" data into new data structures
  - Merge, join, and concatenate data from different dataframes into a single dataframe 
  - Plot data
  
In short, Pandas and its DataFrame object is essentially a one-stop for anything data.

# Learning by Diving In!
Here in this notebook, we'll dive in with some examples and then explain what's going on. The highlights here include:
* Creating DataFrames programmatically and then by reading in data; and in doing so reviewing the structure and key elements of a DataFrame
* Examining key properties and methods of a DataFrame that quickly reveal information about our DataFrame: the number of rows and columns, what types of data it stores, quick summary statistics, etc.
* Summarizing the data in a DataFrame, including how to quickly count and list unique items in a column.
* Examining the various ways to handle missing data in a DataFrame
* Grouping data
* Subsetting observations (rows) and variables (columns)

It all starts by importing the package...

In [1]:
#Import the package
import pandas as pd

## 1. The DataFrame
We'll begin by exploring the key elements of the DataFrame object. Some notions are self evident, i.e., data are stored in rows and columns, much like a spreadsheet. Others are more nuanced: implicit and explicit indices, tables vs. views, and some others.

Let's begin examining the components of DataFrames by examining two ways they can be created.  

#### DataFrame as a list of lists
First, a DataFrame can be considered as a list of lists. Below we see an example where we have 4 sub-lists, each containing 3 items (e.g. the first list [`'Joe'`,`22`,& `True`]). Each of these 4 sub-lists comprises a row in the resulting DataFrame, and each item in a given list becomes a column. 

In [2]:
#Creating a simple data frame as a list of lists
df = pd.DataFrame([['Joe',22,True],
                   ['Bob',25,False],
                   ['Sue',28,False],
                   ['Ken',24,True]],
                  index = [10,20,40,30],
                  columns = ['Name','Age','IsStudent']
                 )
#Display the resulting data frame
df

,Name,Age,IsStudent
10,Joe,22,True
20,Bob,25,False
40,Sue,28,False
30,Ken,24,True


A few key points here: 
* First is that each of the sub-lists has the same number of elements (3) and the same data types as the other sub-lists. Otherwise we'd end up with missing data or "coerced" data types.
* Second is that we also explicitly specify and **index** for the rows (`index = [1,2,3,4]`). The index allows us to identify a specific row.
* Likewise, we explicitly set column names with `columns = ['Name','Age','IsStudent']`, and yes, these allow us to indentify specific columns in our DataFrame.

#### Data frame as a collection of dictionaries
Another way to build (and think of) a DataFrame as a set of dictionaries where each dictionary is a column of data, with the dictionary's key being the column name and it's value being a list of values:

In [3]:
#Creating a data frame as dictionaries of lists
df = pd.DataFrame({"Name":['Joe','Bob','Sue','Ken'],
                   "Age":[22,25,28,24],
                   "IsStudent":[True,False,False,True]},
                  index = [10,20,40,30]
                 )
df

,Name,Age,IsStudent
10,Joe,22,True
20,Bob,25,False
40,Sue,28,False
30,Ken,24,True


While the order of columns has changed (they are alphabetized), the data are the same as the previous DataFrame even though it was created differently. This is just another way to think of the data underlying a DataFrame.

Of note here is that each dictionary has the same number of elements in it and the order of the elements is important otherwise the values in one dictionary/column would lose its proper correspondence with the other elements. 

### Meh, so what...
What does this reveal? List of lists vs set of dictionaries? Well, it explains how you can extract elements from the DataFrame. Thinking of a DataFrame as a list of list, getting the value of the 2nd column, 3rd row is equivalent of getting data from the 2nd item in the 3rd list.  

We can get that value using the DataFrame's `iloc` function (short for intrinsic location), passing the row and column of the location we want.

In [4]:
#Get the 2nd item from the 3rd row; recalling Python is zero-based
df.iloc[2,1]

28

And if we hop over to thinking a DataFrame is a set of dictionaries, we can target a specific value by specifying the index of the value (row) from the dictionary column) we want. The row however, is referred to by the index *we* assigned, not it's implicit index generated by the order in which it was entered. 

In [7]:
#Get the value in the 'Name' column corresponding to the row with an index of '20'
df['Name'][20]

'Bob'

We can also to quick math on our data. If we wanted to calculate the age of our students in days: 

In [8]:
df['Age_days'] = df['Age'] * 365
df

,Name,Age,IsStudent,Age_days
10,Joe,22,True,8030
20,Bob,25,False,9125
40,Sue,28,False,10220
30,Ken,24,True,8760


_We'll return to how we extract data from a DataFrame, but for now just soak in the fact that values in a DataFrame can be referenced by their implicit location (i.e. their row, column coordinates) and by their explicit column name and row index._

---
## 2. Creating a dataframe from a CSV file
More than likely we'll be reading in data vs entering it manually, so let's review how files are read into a Pandas Dataframe. Pandas can read many other formats: Excel files, HTML tables, JSON, etc. But let's concentrate on the simplest one - the csv file - and discuss the key parameters involved. 

In the Data folder within our workspace is a file named `surveys.csv` which holds the data we'll use. If you're curious, this dataset is part of the Portal Teaching data, a subset of the data from Ernst et al [Long-term monitoring and experimental manipulation of a Chihuahuan Desert ecosystem near Portal, Arizona, USA](http://www.esapubs.org/archive/ecol/E090/118/default.htm).

The dataset is stored as a `.csv` file: each row holds information for a single animal, and the columns represent:

| Column | Description |
| :--- | :--- |
|record_id |	Unique id for the observation |
|month| 	month of observation |
|day |	day of observation |
|year |	year of observation |
|plot_id |	ID of a particular plot |
|species_id |	2-letter code |
|sex |	sex of animal (“M”, “F”) |
|hindfoot_length |	length of the hindfoot in mm |
|weight |	weight of the animal in grams |

Below, we read in this file, saving the contents to the variabel `surveys_df`

In [9]:
#Read in the surveys.csv file
surveys_df = pd.read_csv('../data/surveys.csv')

### 2a. Exploring our Species Survey DataFrame
Now, as we often do, let's look at the type of the object we just created.

In [10]:
#Show the object type of the object we just created
type(surveys_df)

pandas.core.frame.DataFrame

As expected, it’s a "DataFrame" (or, to use the full name that Python uses to refer to it internally, a `pandas.core.frame.DataFrame`).

→ What kinds of data does our data frame contain? DataFrames have an attribute called `dtypes` that answers this:

In [11]:
#Show the data types of the columns in our data frame
surveys_df.dtypes

record_id            int64
month                int64
day                  int64
year                 int64
plot_id              int64
species_id          object
sex                 object
hindfoot_length    float64
weight             float64
dtype: object

_All the values in a column have the same type._ For example, months have type `int64`, which is a kind of integer. Cells in the month column cannot have fractional values, but the weight and hindfoot_length columns can, because they have type `float64`. The `object` type doesn’t have a very helpful name, but in this case it represents strings (such as ‘M’ and ‘F’ in the case of sex).

We can print the entire contents of the data frame by just calling the object.

*Remember that in Jupyter notebooks, we can toggle the output by clicking the lightly shaded area to the left of it...*

In [12]:
#Show the data frame's contents
surveys_df

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
...,...,...,...,...,...,...,...,...,...
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN
35546,35547,12,31,2002,10,RM,F,15.0,14.0
35547,35548,12,31,2002,7,DO,M,36.0,51.0


At the bottom of the [long] output above, we see that the data includes 33,549 rows and 9 columns. 

The first column is the **index** of the DataFrame. The index is used to identify the position of the data, but it is not an actual column of the DataFrame. It looks like the `read_csv` function in Pandas read our file properly. 

All the values in a column have the same type. For example, months have type `int64`, which is a kind of integer. Cells in the month column cannot have fractional values, but the weight and hindfoot_length columns can, because they have type `float64`. The `object` type doesn’t have a very helpful name, but in this case it represents strings (such as ‘M’ and ‘F’ in the case of sex).

### 2b. Additional useful DataFrame properties
There are many ways to summarize and access the data stored in DataFrames, using attributes and methods provided by the `DataFrame` object.

To access an <u>attribute</u>, use the DataFrame object name followed by the attribute name `df_object.attribute`. Using the DataFrame `surveys_df` and `attribute` columns, an index of all the column names in the DataFrame can be accessed with `surveys_df.columns`.

<u>Methods</u> are called in a similar fashion using the syntax `df_object.method()`. As an example, `surveys_df.head()` gets the first few rows in the DataFrame `surveys_df` using the `head()` method. With a method, we can supply extra information in the parens to control behaviour.

Let’s look at the data using these.

---
### <font color='red'>*Exercise* - DataFrames</font>
Using our DataFrame `surveys_df`, try out the attributes & methods below to see what they return.
1. `surveys_df.columns`
1. `surveys_df.shape`    Take note of the output of shape - what format does it return the shape of the DataFrame in?
1. `surveys_df.head()`   Also, what does surveys_df.head(15) do?
1. `surveys_df.tail()`

*Use the boxes below to type in the above commands and see what they produce.*

In [13]:
# Challenge 1
surveys_df.columns

Index(['record_id', 'month', 'day', 'year', 'plot_id', 'species_id', 'sex',
       'hindfoot_length', 'weight'],
      dtype='object')

In [14]:
# Challenge 2
surveys_df.shape

(35549, 9)

In [15]:
# Challenge 3
surveys_df.head()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN


In [16]:
# Challenge 4
surveys_df.tail()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN
35546,35547,12,31,2002,10,RM,F,15.0,14.0
35547,35548,12,31,2002,7,DO,M,36.0,51.0
35548,35549,12,31,2002,5,NaN,NaN,NaN,NaN


---
## 3. Inspecting the Data in a Pandas DataFrame
We’ve read our data into Python. Next, let’s perform some quick summary statistics to learn more about the data that we’re working with. We might want to know how many animals were collected in each plot, or how many of each species were caught. We can perform summary stats quickly using groups. But first we need to figure out what we want to group by.

Let’s begin by exploring the data in our data frame:

First, examine the column names. (Yes,I know we just did that in the Challenge above...)

In [17]:
# Look at the column names
surveys_df.columns

Index(['record_id', 'month', 'day', 'year', 'plot_id', 'species_id', 'sex',
       'hindfoot_length', 'weight'],
      dtype='object')

We can extract one column of data into a new object by referencing that column as shown here:

In [18]:
speciesIDs = surveys_df['species_id']

Examining the type of this `speciesIDs` object reveals another Pandas data type: the *Series* which is slightly different than a DataFrame...

In [19]:
type(speciesIDs)

pandas.core.series.Series

A `series` object is a one-dimensional array, much like a NumPy array, with its own set of properties and functions. The values are indexed allowing us to extract values at a specific row (try: `speciesIDs[5]`) or slice of rows (try: `species[2:7]`). 

We can also, using the `series.nunique()` and `series.unique()` functions, generate a count of unique values in the series and a list of unique values, respectively. 

In [20]:
#Reveal how many unique species_ID values are in the table
speciesIDs.nunique()

48

In [21]:
#List the unique values
speciesIDs.unique()

array(['NL', 'DM', 'PF', 'PE', 'DS', 'PP', 'SH', 'OT', 'DO', 'OX', 'SS',
       'OL', 'RM', nan, 'SA', 'PM', 'AH', 'DX', 'AB', 'CB', 'CM', 'CQ',
       'RF', 'PC', 'PG', 'PH', 'PU', 'CV', 'UR', 'UP', 'ZL', 'UL', 'CS',
       'SC', 'BA', 'SF', 'RO', 'AS', 'SO', 'PI', 'ST', 'CU', 'SU', 'RX',
       'PB', 'PL', 'PX', 'CT', 'US'], dtype=object)

---
### <font color='red'>*Challenge* - Counts and Lists from Data </font>


1. Create a list of unique plot ID’s found in the surveys data (much like we did above with the species id values). Call it `plot_names`. How many unique plots are there in the data? How many unique species are in the data?

1. What is the difference between `len(plot_names)` and `surveys_df['plot_id'].nunique()`?


In [22]:
# Challenge 1
PLOT_IDs=surveys_df['plot_id']

In [26]:
# Challenge 2
surveys_df[plot_id.nunique()]

NameError: name 'plot_id' is not defined

In [24]:
PLOT_IDs.unique()

array([ 2,  3,  7,  1,  6,  5,  8,  4, 11, 14, 15, 13,  9, 10, 17, 16, 20,
       23, 18, 21, 22, 19, 12, 24], dtype=int64)

---

## 4. Grouping Data in Pandas
We often want to calculate summary statistics grouped by subsets or attributes within fields of our data. For example, we might want to calculate the average weight of all individuals _per plot_.

We can calculate basic statistics for all records in a single column using the syntax below:

In [27]:
surveys_df['weight'].describe()

count    32283.000000
mean        42.672428
std         36.631259
min          4.000000
25%         20.000000
50%         37.000000
75%         48.000000
max        280.000000
Name: weight, dtype: float64

We can also extract one specific metric if we wish:

In [28]:
print(" Min: ", surveys_df['weight'].min())
print(" Max: ", surveys_df['weight'].max())
print(" Mean: ", surveys_df['weight'].mean())
print(" Std Dev: ", surveys_df['weight'].std())
print(" Count: ", surveys_df['weight'].count())

 Min:  4.0
 Max:  280.0
 Mean:  42.672428212991356
 Std Dev:  36.63125947458358
 Count:  32283


But if we want to summarize by one or more variables, for example sex, we can use Pandas’ `.groupby` method. Once we’ve created a groupby DataFrame, we can quickly calculate summary statistics by a group of our choice.

In [29]:
# Group data by sex
grouped_data = surveys_df.groupby('sex')

In [30]:
# Show just the grouped means
grouped_data.mean()

,record_id,month,day,year,plot_id,hindfoot_length,weight
sex,,,,,,,
F,18036.412046,6.583047,16.007138,1990.644997,11.440854,28.836780,42.170555
M,17754.835601,6.392668,16.184286,1990.480401,11.098282,29.709578,42.995379


In [31]:
# Or, use the describe function to reveal all summary stats for the grouped data
grouped_data.describe()

record_id                                                               \
        count          mean           std  min      25%      50%       75%   
sex                                                                          
F     15690.0  18036.412046  10423.089000  3.0  8917.50  18075.5  27250.00   
M     17348.0  17754.835601  10132.203323  1.0  8969.75  17727.5  26454.25   

                month            ... hindfoot_length         weight  \
         max    count      mean  ...             75%   max    count   
sex                              ...                                  
F    35547.0  15690.0  6.583047  ...            36.0  64.0  15303.0   
M    35548.0  17348.0  6.392668  ...            36.0  58.0  16879.0   

                                                         
          mean        std  min   25%   50%   75%    max  
sex                                                      
F    42.170555  36.847958  4.0  20.0  34.0  46.0  274.0  
M    42.995379  36.184981  4.0  20.0  39.0  49.0  280.0  

[2 rows x 56 columns]

---
### <font color='red'>Challenge - Summary Data </font>
1. How many recorded individuals are female `F` and how many male `M`

In [32]:
# Challenge 1: Show the *count* of records, grouped by sex
grouped_data.count()

,record_id,month,day,year,plot_id,species_id,hindfoot_length,weight
sex,,,,,,,,
F,15690,15690,15690,15690,15690,15690,14894,15303
M,17348,17348,17348,17348,17348,17348,16476,16879


2. The statement above produces another dataframe. Show just the count of the records in the `weight` field in the resulting dataframe.

In [ ]:
# Challege 2: Just show the 'weight' column in the above statement
grouped_data.█()['█']

3. What happens when you group by two columns using the following syntax and then grab mean values:
 * `grouped_data2 = surveys_df.groupby(['plot_id','sex'])`
 * `grouped_data2.mean()`

In [ ]:
# Challenge 3


4. Display a summary of **hindfood length** values for each **plot id** in your data. _HINT: First group data, then extract the one variable you want to summarize, and finally use the `describe` function on that result to compute summary statistics..._

In [ ]:
# Challenge 4


### *More complex aggregating functions...*
We can also supply a **dictionary of aggregating functions** so that each column in the grouped result is aggregated exactly how we want (i.e. instead of computing just the sum or mean of all columns). This dictionary is built by specifying the <u>column name as the key</u> and the <u>aggregate function(s) as the values</u>. Below is an example to aggregate the data by `sex`, computing the minumum and maximum of the `year`, the median of the `hindfoot_length`, and the mean `weight`.

In [33]:
#Create the aggregate function dictionary
aggFuncs = {
    "year": ['count','min','max'],#compute count, min, and max of the year attribute
    "hindfoot_length": 'median',  #compute the median hindfoot length
    "weight": 'mean'}             #compute the mean weight
#Apply the dictionary
surveys_df.groupby('sex').agg(aggFuncs)

year             hindfoot_length     weight
     count   min   max          median       mean
sex                                              
F    15690  1977  2002            27.0  42.170555
M    17348  1977  2002            34.0  42.995379